<a href="https://colab.research.google.com/github/fininsight/page-election/blob/master/kshong/%EC%A2%85%EB%A1%9C%EA%B5%AC_%EB%AF%BC%EC%9B%90_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 23), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (60/60), done.


In [0]:
 cd Mecab-ko-for-Google-Colab

/content/Mecab-ko-for-Google-Colab


In [0]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 226kB/s 
     |████████████████████████████████| 2.4MB 48.2MB/s 
     |████████████████████████████████| 92kB 8.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-03-09 06:22:20--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.0, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?S

In [0]:
from google.colab import files

In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving jongro_minwon.xlsx to jongro_minwon.xlsx
Saving jongro_minwon2_2.xlsx to jongro_minwon2_2.xlsx
User uploaded file "jongro_minwon.xlsx" with length 277921 bytes
User uploaded file "jongro_minwon2_2.xlsx" with length 3842421 bytes


In [0]:
from tqdm import tqdm_notebook # progress bar
from konlpy.tag import Mecab #Mecab, Okt 등 형태소 분석기 불러오기
import numpy as np
import string # 특수문자
import re
import warnings # 경고 알림 제거
import pickle
from gensim import corpora
from gensim import models
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

In [0]:
def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)

    return docs

In [0]:
def read_documents(filename, type):
    corpus = []
    df = pd.read_excel(filename)
    df = df.replace(np.nan, '', regex=True)
    for i, row in df.iterrows():
            if type == 'all':
                title = row['title']
                if title:
                    corpus.append(title)
            
            content = row['content'] 
            if content:
                corpus.append(content)
                    
    
    print(len(corpus))
    return corpus

In [0]:
def text_tokenizing(corpus, tokenizer):
    
    mecab = Mecab()
    token_corpus = []
    

    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if len(word) > 1]
                
            token_corpus.append(token_text)
            
    elif tokenizer == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if len(word) > 1]
            token_corpus.append(token_text)
        

    return token_corpus

In [0]:
def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

In [0]:
NUM_TOPIC_WORDS = 10
NUM_TOPICS = 6
files = ['jongro_minwon.xlsx', 'jongro_minwon2_2.xlsx']
documents = []
for file in files:
    documents += read_documents(file, 'content')

cleaned_text = text_cleaning(documents)

tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun")

dictionary = corpora.Dictionary(tokenized_text)
corpus = [dictionary.doc2bow(text) for text in tokenized_text]
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
model = models.ldamodel.LdaModel(corpus_tfidf, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")

print_topic_words(model)

792
9113




Printing topic words.

Topic ID: 0
	태극기	0.0022015138529241085
	개똥	0.0015069840010255575
	사람	0.0014575622044503689
	공원	0.0014330241829156876
	예약	0.0013591608731076121
	민원	0.001316947746090591
	단속	0.001308835344389081
	부탁	0.0012828956823796034
	건널목	0.001259488402865827
	구청	0.001229256740771234


Topic ID: 1
	쓰레기	0.00692339800298214
	단속	0.0054689571261405945
	불법	0.0035679282154887915
	사람	0.0034862810280174017
	주차	0.0033181628677994013
	수거	0.0032927305437624454
	버스	0.003077705390751362
	차량	0.002774375258013606
	처리	0.0027702045626938343
	골목	0.00264382385648787


Topic ID: 2
	공사	0.009111053310334682
	소음	0.005703998729586601
	주차	0.004197102505713701
	건물	0.003939300309866667
	민원	0.0038940394297242165
	소리	0.0032896308694034815
	저희	0.0032607284374535084
	전화	0.0030847988091409206
	구청	0.002882228000089526
	건축	0.0028545758686959743


Topic ID: 3
	처리	0.0024490852374583483
	현수막	0.002238641493022442
	불법	0.0021143953781574965
	광고	0.002105891238898039
	민원	0.0020985666196793318
	답변	0.001994743943214416

In [0]:
!pip install pyldavis

     |████████████████████████████████| 1.6MB 5.0MB/s 
     |████████████████████████████████| 552kB 40.6MB/s 
  Created wheel for pyldavis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=bd08f8f5135faa7f47a3b2d827b6dc122a620d02c59b9708ce36db088258a574
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=3d5cc141ac4aa0466db38439aff0fe51768e7d360e113a582170754c86702c1f
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyldavis funcy


In [0]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

/usr/local/lib/python3.6/dist-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.075030  0.028102       1        1  41.444069
1     -0.017937 -0.065013       2        1  36.415897
5      0.021033 -0.026101       3        1  11.416383
3      0.018411  0.026651       4        1   7.429921
4      0.021888  0.015346       5        1   1.932847
0      0.031636  0.021015       6        1   1.360891, topic_info=    Category         Freq Term        Total  loglift  logprob
470  Default  1930.000000  쓰레기  1930.000000  30.0000  30.0000
314  Default  2959.000000   공사  2959.000000  29.0000  29.0000
18   Default  2269.000000   주차  2269.000000  28.0000  28.0000
36   Default  1354.000000   도로  1354.000000  27.0000  27.0000
20   Default  1552.000000   차량  1552.000000  26.0000  26.0000
..       ...          ...  ...          ...      ...      ...
46    Topic6     7.869813   불법  1482.226196  -0.9412  -6.9628
18    Topic6     7.880630   주차  2269.587646  -1.3659  -6.9614
470   Topic6     7.732556  쓰레기  1930.672363  -1.2232  -6.9804
314   Topic6     7.745633   공사  2959.804932  -1.6487  -6.9787
312   Topic6     7.638384   건물  1648.538086  -1.0774  -6.9926

[501 rows x 6 columns], token_table=       Topic      Freq Term
term                       
18930      1  0.161955   表箋
18930      2  0.161955   表箋
18930      3  0.161955   表箋
18930      4  0.161955   表箋
18930      5  0.323910   表箋
...      ...       ...  ...
666        2  0.820014  흡연자
666        3  0.038784  흡연자
666        4  0.022163  흡연자
666        5  0.011081  흡연자
666        6  0.016622  흡연자

[1926 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 6, 4, 5, 1])

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
print('test')